****
# Algorithmic Problems solved in Python 3.7 (cont.)
****
<p style="text-align: right"><i>Jesus Perez Colino<br>First version: November 2018<br></i></p>

## About this notebook: 
****
Notebook prepared by **Jesus Perez Colino** Version 0.2, First Released: 25/11/2018, Alpha

- This work is licensed under a [Creative Commons Attribution-ShareAlike 3.0 Unported License](http://creativecommons.org/licenses/by-sa/3.0/deed.en_US). This work is offered for free, with the hope that it will be useful.


- **Summary**: 

>This notebook is a **continuation** of the previous notebooks in algorithmic problems, but this time for Python 3.7. Some of them are well-know in Computer Science (searching, sorting, recursion, dynamic programming...), and other are more functional programming oriented or mathematically based. It includes examples of list comprenhension, map/reduce functions, generators, decorators, HPC simulations...


- **Index**:


    Problem 1: Is Fibonacci? (... starting with a classic)
    Problem 2: Charles Babbage's homage
    Problem 3: Dynamic programming 
    Problem 4: Knapsack problem (another classical)
    Problem 5: Customizing String Formatting
    Problem 6: Implementing a Data Model or Type System using Descriptors
    Problem 7: Calling a Method on an Object given the Name as a string
    Problem 8: Making Classes Support Comparison Operations Problem
    Problem 9: Implementing a Priority Queue
    Problem 10: Finding the Largest and the Smallest
    Problem 11: Keeping the Last N Items
    Problem 12: Mapping Keys to Multiple Values in a Dictionary
    Problem 13: Keeping Dictionaries in Order
    Problem 14: Calculating with dictionaries
    Problem 15: Finding Commonalities in Two Dictionaries
    Problem 16: Removing Duplicates from a Sequence while Maintaining Order
    Problem 17: Determining the Most Frequently Occurring Items in a Sequence
    Problem 18: Sorting a List of Dictionaries by a Common Key
    Problem 19: Sorting Objects Without Native Comparison Support
    Problem 20: Grouping Records Together Based on a Field
    Problem 21: Filtering Sequence Elements
    Problem 22: Test for Palindromic Permutations
    Problem 23: Compute the Intersection of Two Sorted Arrays
    Problem 24: Kadene's algorithm (... another classic)
    Problem 25: Merge two Sorted Lists
    Problem 26: Root Square of the Conditional Values of a dictionary
    Problem 27: More about Palindromes...
    Problem 28: Reversing integers
    Problem 29: Find two elements that sum equal the target...
    Problem 30: Count the number of ways to traverse a 2D array
    Problem 31: Find the Longest Common Prefix


In [1]:
import IPython
import numpy as np
np.random.seed(12345)
import pandas as pd
import matplotlib.pyplot as plt
import watermark

import warnings
warnings.filterwarnings('ignore')

%load_ext watermark
%matplotlib inline

print(' Reproducibility conditions for this notebook '.center(85,'-'))
%watermark -n -v -m -p numpy,scipy,matplotlib,pandas
print('-'*85)

-------------------- Reproducibility conditions for this notebook -------------------
Mon Dec 09 2019 

CPython 3.7.4
IPython 7.8.0

numpy 1.16.5
scipy 1.3.1
matplotlib 3.1.1
pandas 0.25.1

compiler   : MSC v.1915 64 bit (AMD64)
system     : Windows
release    : 10
machine    : AMD64
processor  : Intel64 Family 6 Model 94 Stepping 3, GenuineIntel
CPU cores  : 8
interpreter: 64bit
-------------------------------------------------------------------------------------


<a id=Problem 1: Is Fibonacci?><\a>

## Problem 1: Is Fibonacci?

Given a list of integers `mylist`, write a program to determine if any of the elements in `mylist` is an element of the Fibonacci sequence.

In [33]:
def solve(mylist):
    
    fib_list = [0, 1]
    fib_next = fib_list[-2] + fib_list[-1]
    
    while fib_next < max(mylist)+1:
        fib_list.append(fib_next)
        fib_next =  fib_list[-2] + fib_list[-1]
        
    for element in mylist:
        if element in fib_list: print('{} is in the Fib. sequence'.format(element))
        else: print('{} is not in the Fib. sequence'.format(element))
            
    return

In [34]:
my_list = [2,3,4,5,6,7,8,34]
solve(my_list)

2 is in the Fib. sequence
3 is in the Fib. sequence
4 is not in the Fib. sequence
5 is in the Fib. sequence
6 is not in the Fib. sequence
7 is not in the Fib. sequence
8 is in the Fib. sequence
34 is in the Fib. sequence


In [22]:
# just for fun... here the simplest version to obtain the Fibonacci sequence
# Notice that this version is not storing the whole sequence

def fib(n):
    if n <= 1: return n
    fib_minus_2, fib_minus_1 = 0, 1
    for _ in range(2,n):
        fib_next = fib_minus_2 + fib_minus_1
        fib_minus_2, fib_minus_1 = fib_minus_1 , fib_next
    return fib_next

fib(10)


34

In [35]:
# and also, more fun... Fibonacci sequence in one line:

from functools import reduce

fib = lambda n : reduce(lambda fib , n : [ fib[1] , fib[0] + fib[1] ], range(n-1) ,[0,1])[0]

for i in range(10):
    print(f'{i+1} element of fib. seq. is {fib(i+1)}')



1 element of fib. seq. is 0
2 element of fib. seq. is 1
3 element of fib. seq. is 1
4 element of fib. seq. is 2
5 element of fib. seq. is 3
6 element of fib. seq. is 5
7 element of fib. seq. is 8
8 element of fib. seq. is 13
9 element of fib. seq. is 21
10 element of fib. seq. is 34


## Problem 2: Charles Babbage's homage

Charles Babbage (1791 - 1871) loooking ahead to the sorts of problems his Analytical Engine would be able to solve, gave this example:
> What's the smallest positive integer whose square ends in the digits 269,696?

He thought the answer might be 99,736, whose square is 9,947,269,696 but he could not be certain...

In [16]:
x = [x for x in range(100000) if (x*x) % 1000000 == 269696][0]
print(x, 'squared is equal to', x*x, 'and it ends in ', str(x*x)[-6:])

25264 squared is equal to 638269696 and it ends in  269696


In [75]:
x = [x for x in range(100000) if str(x*x)[-6:]== '269696'][0]
print(x, 'squared is equal to', x*x, 'and it ends in ', str(x*x)[-6:])

25264 squared is equal to 638269696 and it ends in  269696


## Problem 3: Dynamic programming

Find the number of sets of integers in `[ 2, 4, 6, 10 ]` that add up to `16`

In [53]:
# First solution: recursive version (no optimal)

def recursive(array, total, i): 
    if total == 0 : 
        return 1
    elif total < 0 : 
        return 0
    elif i < 0 : 
        return 0
    elif total < array[i]: 
        return recursive(array, total, i-1)
    else: 
        return recursive(array, total - array[i], i-1) + recursive(array, total, i-1)
    

def count_set(array, total):
    return recursive(array, total, len(array)-1)


In [56]:
myarray = [2,4,6,10]
total_sum = 16

In [63]:

count_set(myarray, total_sum)

2

In [50]:
# Second solution: dynamic programming solution (memoizing)

def count_sets_dp(array, total):
    mem = {}
    return dp(array, total, len(array)-1, mem)

def dp(array, total, i, mem):
    key = str(total) + ':' + str(i)
    if key in mem:
        return mem[key]
    elif total == 0:
        return 1
    elif total < 0:
        return 0
    elif i < 0:
        return 0
    elif total < array[i]:
        to_return = dp(array, total, i-1, mem)
    else: 
        to_return = dp(array, total-array[i], i-1, mem) \
                    + dp(array, total, i-1, mem)
        
    mem[key] = to_return
    return to_return


In [62]:

count_sets_dp(a, total_sum)    


2

## Problem 4: Knapsack problem (another classical)

Given a set of items, each with a weight and a value, determine the number of each item to include in a collection, so that the total weight is less or equal to a given limit and the total value is as larger as possible.

In [17]:
# list of items: tuple of tuples of ('name', weight, value)

items = (
    ("map", 9, 150), ("compass", 13, 35), ("water", 153, 200), ("sandwich", 50, 160),
    ("glucose", 15, 60), ("tin", 68, 45), ("banana", 27, 60), ("apple", 39, 40),
    ("cheese", 23, 30), ("beer", 52, 10), ("suntan cream", 11, 70), ("camera", 32, 30),
    ("t-shirt", 24, 15), ("trousers", 48, 10), ("umbrella", 73, 40),
    ("waterproof trousers", 42, 70), ("waterproof overclothes", 43, 75),
    ("note-case", 22, 80), ("sunglasses", 7, 20), ("towel", 18, 12),
    ("socks", 4, 50), ("book", 30, 10),
    )

max_weight = 400

In [18]:
def total_value(items, max_weight):
    return sum([x[2] for x in items]) if sum([x[1] for x in items])<max_weight else 0

In [45]:
# Recursive dynamic algorithmic solution

cache = {}
def solve(items, max_weight):
    if not items:
        return()
    if (items, max_weight) not in cache:
        head = items[0]
        tail = items[1:]
        include = (head,) + solve(tail, max_weight - head[1])
        dont_include = solve(tail, max_weight)
        if total_value(include, max_weight) > total_value(dont_include, max_weight):
            answer = include
        else:
            answer = dont_include
        cache[(items, max_weight)] = answer
    return cache[(items, max_weight)]



In [47]:
cache = {}
solution = solve(items, max_weight)

print('items: ')
for x in solution: print(x[0])
print('='*20)
print('value: ', total_value(solution, max_weight))
print('weight: ', sum([x[1] for x in solution]))
print('='*20)

items: 
map
compass
water
sandwich
glucose
banana
suntan cream
waterproof trousers
waterproof overclothes
note-case
sunglasses
socks
value:  1030
weight:  396


## Problem 5: Customizing String Formatting
Build a Date class, with year, month and day as inputs, that is able to return the date in multiple formats, using an external format dictionary.

In [62]:
_formats = {
    'ymd' : '{d.year}--{d.month}--{d.day}',
    'mdy' : '{d.month}//{d.day}//{d.year}',
    'dmy' : '{d.day}||{d.month}||{d.year}'
    }

In [63]:
class Date(object):
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day
    def __format__(self, code):
        if code == '':
            code = 'ymd'
        fmt = _formats[code]
        return fmt.format(d=self)
    def __str__(self):
        return format(d,'ymd')

In [64]:
d = Date(2012, 12, 21)
print(d)

2012--12--21


In [65]:
f"The date is {format(d)}"

'The date is 2012--12--21'

In [66]:
f"The date is {format(d, 'mdy')}"

'The date is 12//21//2012'

In [67]:
f"The date is {format(d, 'dmy')}"


'The date is 21||12||2012'

## Problem 6: Implementing a Data Model or Type System using Descriptors

You want to define several kinds of data structures, but want to enforce constraints on the values that are allowed to be assigned to certain attributes.

In [88]:
# Base class. Uses a descriptor to set a value

class Descriptor(object):
    def __init__(self, name=None, **opts):
        self.name = name
        for key, value in opts.items():
            setattr(self, key, value)
    def __set__(self, instance, value):
        instance.__dict__[self.name] = value

In [169]:
# Descriptor for enforcing types

class Typed(Descriptor):
    expected_type = type(None)
    def __set__(self, instance, value):
        if not isinstance(value, self.expected_type):
            raise TypeError('expected ' + str(self.expected_type))
        super().__set__(instance, value)

In [90]:
class Unsigned(Descriptor):
    def __set__(self, instance, value):
        if value < 0:
            raise ValueError('Expected >= 0')
        super().__set__(instance, value)

In [91]:
class MaxSized(Descriptor):
    def __init__(self, name=None, **opts):
        if 'size' not in opts:
            raise TypeError('missing size option')
        super().__init__(name, **opts)
    def __set__(self, instance, value):
        if len(value) >= self.size:
            raise ValueError('size must be < ' + str(self.size))
        super().__set__(instance, value)

These classes should be viewed as basic building blocks from which you construct a data model or type system. 

Continuing, here is some code that implements some different kinds of data:

In [92]:
class Integer(Typed):
    expected_type = int

class UnsignedInteger(Integer, Unsigned):
    pass

class Float(Typed):
    expected_type = float

class UnsignedFloat(Float, Unsigned):
    pass

class String(Typed):
    expected_type = str

class SizedString(String, MaxSized):
    pass

In [158]:
class Stock(list):
    # Specify constraints
    name = SizedString('name',size=8)
    shares = UnsignedInteger('shares')
    price = UnsignedFloat('price')
    def __init__(self, name, shares, price):
        self.name = name
        self.shares = shares
        self.price = price
    def __str__(self):
        return f"{[self.name, self.shares, self.price]}"


In [162]:
s = Stock('GOOG', 10, 11.1)
print(s)


['GOOG', 10, 11.1]


In [167]:
s = Stock(11.1, 11.1, 11.1)

print(s) # <<<--- It should print an Type ERROR 

TypeError: expected <class 'str'>

Another approach is to use a class decorator, like this:

In [166]:
# Class decorator to apply constraints
def check_attributes(**kwargs):
    def decorate(cls):
        for key, value in kwargs.items():
            if isinstance(value, Descriptor):
                value.name = key
                setattr(cls, key, value)
            else:
                setattr(cls, key, value(key))
        return cls
    return decorate

In [166]:
# Example
@check_attributes(name=SizedString(size=8),
                  shares=UnsignedInteger,
                  price=UnsignedFloat)
class Stock(object):
    def __init__(self, name, shares, price):
        self.name = name
        self.shares = shares
        self.price = price

Or, with an additional twist, a class decorator approach can also be used as a replacement for mixin classes, multiple inheritance, and tricky use of the super() function. Here is an alternative formulation of this recipe that uses class decorators:

In [171]:
# Base class. Uses a descriptor to set a value
class Descriptor(object):
    def __init__(self, name=None, **opts):
        self.name = name
        for key, value in opts.items():
            setattr(self, key, value)
    def __set__(self, instance, value):
        instance.__dict__[self.name] = value

In [172]:
# Decorator for applying type checking
def Typed(expected_type, cls=None):
    if cls is None:
        return lambda cls: Typed(expected_type, cls)
    super_set = cls.__set__
    def __set__(self, instance, value):
        if not isinstance(value, expected_type):
            raise TypeError('expected ' + str(expected_type))
        super_set(self, instance, value)
    cls.__set__ = __set__
    return cls

# Decorator for unsigned values
def Unsigned(cls):
    super_set = cls.__set__
    def __set__(self, instance, value):
        if value < 0:
            raise ValueError('Expected >= 0')
        super_set(self, instance, value)
    cls.__set__ = __set__
    return cls

# Decorator for allowing sized values
def MaxSized(cls):
    super_init = cls.__init__
    def __init__(self, name=None, **opts):
        if 'size' not in opts:
            raise TypeError('missing size option')
        super_init(self, name, **opts)
    cls.__init__ = __init__
    super_set = cls.__set__
    def __set__(self, instance, value):
        if len(value) >= self.size:
            raise ValueError('size must be < ' + str(self.size))
        super_set(self, instance, value)
    cls.__set__ = __set__
    return cls

In [173]:
# Specialized descriptors

@Typed(int)
class Integer(Descriptor):
    pass

@Unsigned
class UnsignedInteger(Integer):
    pass

@Typed(float)
class Float(Descriptor):
    pass

@Unsigned
class UnsignedFloat(Float):
    pass

@Typed(str)
class String(Descriptor):
    pass

@MaxSized
class SizedString(String):
    pass

## Problem 7: Calling a Method on an Object Given the Name As a String

You have the name of a method that you want to call on an object stored in a string and you want to execute the method.

In [175]:
import math

class Point(object):
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __repr__(self):
        return 'Point({!r:},{!r:})'.format(self.x, self.y)
    def distance(self, x, y):
        return math.hypot(self.x - x, self.y - y)

In [182]:
p = Point(2, 3)
p.distance(0,0)       # <- Usual call of method

3.605551275463989

In [180]:
d = getattr(p, 'distance')(0, 0)     # <- Calls p.distance(0, 0) as string
d

3.605551275463989

In [181]:
getattr(p, 'distance')(2, 3)     # Calls p.distance(0, 0)

0.0

## Problem 8: Making Classes Support Comparison Operations Problem

You’d like to be able to compare instances of your class using the standard comparison operators (e.g., >=, !=, ⇐, etc.), but without having to write a lot of special methods.

In [189]:
from functools import total_ordering
class Room(object):
    def __init__(self, name, length, width):
        self.name = name
        self.length = length
        self.width = width
        self.square_feet = self.length * self.width

@total_ordering
class House(object):
    def __init__(self, name, style):
        self.name = name
        self.style = style
        self.rooms = list()
    @property
    def living_space_footage(self):
        return sum(r.square_feet for r in self.rooms)
    def add_room(self, room):
        self.rooms.append(room)
    def __str__(self):
        return '{}: {} square foot {}'.format(self.name, self.living_space_footage, self.style)
    def __eq__(self, other):
        return self.living_space_footage == other.living_space_footage
    def __lt__(self, other):
        return self.living_space_footage < other.living_space_footage


In [190]:
# Build a few houses, and add rooms to them
h1 = House('house1', 'Cape')
h1.add_room(Room('Master Bedroom', 14, 21))
h1.add_room(Room('Living Room', 18, 20))
h1.add_room(Room('Kitchen', 12, 16))
h1.add_room(Room('Office', 12, 12))

h2 = House('house2', 'Ranch')
h2.add_room(Room('Master Bedroom', 14, 21))
h2.add_room(Room('Living Room', 18, 20))
h2.add_room(Room('Kitchen', 12, 16))

h3 = House('house3', 'Split')
h3.add_room(Room('Master Bedroom', 14, 21))
h3.add_room(Room('Living Room', 18, 20))
h3.add_room(Room('Office', 12, 16))
h3.add_room(Room('Kitchen', 15, 17))
houses = [h1, h2, h3]

In [191]:
print('Is house1 bigger than house2?: ', h1 > h2)

Is house1 bigger than house2?:  True


In [192]:
print('Is house2 smaller than house3?: ', h2 < h3) 

Is house2 smaller than house3?:  True


In [193]:
print('Is house2 greater than or equal to house1?', h2 >= h1) 

Is house2 greater than or equal to house1? False


## Problem 9: Implementing a Priority Queue

Implement a queue that **sorts items by a given priority**, and always **returns the item with the highest priority** on each pop operation

In [7]:
import heapq

class PriorityQueue(object):
    def __init__(self):
        self._queue = []
        self._index = 0
    def push(self, item, priority):
        heapq.heappush(self._queue, (-priority, self._index, item))
        self._index += 1
    def pop(self):
        return heapq.heappop(self._queue)[-1]
    def __repr__(self):
        return f"queue element: {self._queue} index: {self._index}"

In [8]:
class Item(object):
    def __init__(self, name):
        self.name = name
    def __repr__(self):
        return 'Item({!r})'.format(self.name)

In [31]:
q = PriorityQueue()
q.push(Item('foo'), 1)
q

queue element: [(-1, 0, Item('foo'))] index: 1

In [32]:
q.push(Item('bar'), 5)
q

queue element: [(-5, 1, Item('bar')), (-1, 0, Item('foo'))] index: 2

In [33]:
q.push(Item('spam'), 4)
q

queue element: [(-5, 1, Item('bar')), (-1, 0, Item('foo')), (-4, 2, Item('spam'))] index: 3

In [34]:
q.push(Item('grok'), 1)
q

queue element: [(-5, 1, Item('bar')), (-1, 0, Item('foo')), (-4, 2, Item('spam')), (-1, 3, Item('grok'))] index: 4

In [35]:
print(q.pop())
print(q.pop())
print(q.pop())
print(q.pop())

Item('bar')
Item('spam')
Item('foo')
Item('grok')


## Problem 10: Finding the Largest and the Smallest

Make a list of the largest or smallest N items in a collection.

In [36]:
portfolio = [
   {'name': 'IBM', 'shares': 100, 'price': 91.1},
   {'name': 'AAPL', 'shares': 50, 'price': 543.22},
   {'name': 'FB', 'shares': 200, 'price': 21.09},
   {'name': 'HPQ', 'shares': 35, 'price': 31.75},
   {'name': 'YHOO', 'shares': 45, 'price': 16.35},
   {'name': 'ACME', 'shares': 75, 'price': 115.65}
]

In [40]:
import heapq

cheap = heapq.nsmallest(3, portfolio, key=lambda s: s['price'])
cheap

[{'name': 'YHOO', 'shares': 45, 'price': 16.35},
 {'name': 'FB', 'shares': 200, 'price': 21.09},
 {'name': 'HPQ', 'shares': 35, 'price': 31.75}]

In [41]:
expensive = heapq.nlargest(3, portfolio, key=lambda s: s['price'])
expensive

[{'name': 'AAPL', 'shares': 50, 'price': 543.22},
 {'name': 'ACME', 'shares': 75, 'price': 115.65},
 {'name': 'IBM', 'shares': 100, 'price': 91.1}]

## Problem 11: Keeping the Last N Items

Keep a limited history of the last few items seen during iteration or during some other kind of processing

In [42]:
from collections import deque

q = deque(maxlen=3)
q.append(1)
q.append(2)
q.append(3)
q

deque([1, 2, 3])

In [44]:
q.append(10)
q.append(20)
q.append(30)
q

deque([10, 20, 30])

In [45]:
def search(lines, pattern, history=5):
    previous_lines = deque(maxlen=history)
    for line in lines:
        if pattern in line:
            yield line, previous_lines
        previous_lines.append(line)

In [ ]:
with open('somefile.txt') as f:
    for line, prevlines in search(f, 'python', 5):
        for pline in prevlines:
            print(pline, end='')
        print(line, end='')
        print('-'*20)

## Problem 12: Mapping Keys to Multiple Values in a Dictionary

You want to make a dictionary that maps keys to more than one value (a so-called "multidict")

In [47]:
# A dictionary is a mapping where each key is mapped to a single value. 
# If you want to map keys to multiple values, you need to store the multiple 
# values in another container such as a list or set. 

d = {
   'a' : [1, 2, 3],
   'b' : [4, 5]
}

e = {
   'a' : {1, 2, 3},
   'b' : {4, 5}
}

In [48]:
from collections import defaultdict

d = defaultdict(list)
d['a'].append(1)
d['a'].append(2)
d['b'].append(4)

d

defaultdict(list, {'a': [1, 2], 'b': [4]})

In [49]:
d = defaultdict(set)
d['a'].add(1)
d['a'].add(2)
d['b'].add(4)

d

defaultdict(set, {'a': {1, 2}, 'b': {4}})

In [51]:
d = {}    # A regular dictionary
d.setdefault('a', []).append(1)
d.setdefault('a', []).append(2)
d.setdefault('b', []).append(4)
d

{'a': [1, 2], 'b': [4]}

## Problem 13: Keeping Dictionaries in Order

Create a dictionary, and you also want to control the order of items when iterating or serializing

In [53]:
#To control the order of items in a dictionary, you can use an OrderedDict from the collections module. 
#It exactly preserves the original insertion order of data when iterating.

from collections import OrderedDict

d = OrderedDict()
d['foo'] = 1
d['bar'] = 2
d['spam'] = 3
d['grok'] = 4

# Outputs "foo 1", "bar 2", "spam 3", "grok 4"
for key in d:
    print(key, d[key])

foo 1
bar 2
spam 3
grok 4


## Problem 14: Calculating with dictionaries

Perform various calculations (e.g., minimum value, maximum value, sorting, etc.) on a dictionary of data

In [54]:
prices = {
   'ACME': 45.23,
   'AAPL': 612.78,
   'IBM': 205.55,
   'HPQ': 37.20,
   'FB': 10.75
}

In [65]:
min(prices, key=lambda k: prices[k])  

'FB'

In [66]:
max(prices, key=lambda k: prices[k]) 

'AAPL'

In [55]:
min_price = min(zip(prices.values(), prices.keys()))
min_price

(10.75, 'FB')

In [57]:
max_price = max(zip(prices.values(), prices.keys()))
max_price

(612.78, 'AAPL')

In [58]:
prices_sorted = sorted(zip(prices.values(), prices.keys()))
prices_sorted

[(10.75, 'FB'),
 (37.2, 'HPQ'),
 (45.23, 'ACME'),
 (205.55, 'IBM'),
 (612.78, 'AAPL')]

In [64]:
x = list(zip(prices.values(), prices.keys()))
print(x)

[(45.23, 'ACME'), (612.78, 'AAPL'), (205.55, 'IBM'), (37.2, 'HPQ'), (10.75, 'FB')]


## Problem 15: Finding Commonalities in Two Dictionaries

You have two dictionaries and want to find out what they might have in common (same keys, same values, etc.).

In [67]:
a = {
   'x' : 1,
   'y' : 2,
   'z' : 3
}

b = {
   'w' : 10,
   'x' : 11,
   'y' : 2
}

In [72]:
# Find keys in common
a.keys() & b.keys()

{'x', 'y'}

In [71]:
# Find keys in a that are not in b
a.keys() - b.keys() 

{'z'}

In [70]:
# Find (key,value) pairs in common
a.items() & b.items()

{('y', 2)}

In [73]:
# Make a new dictionary with certain keys removed
c = {key:a[key] for key in a.keys() - {'z', 'w'}}
# c is {'x': 1, 'y': 2}
c

{'x': 1, 'y': 2}

## Problem 16: Removing Duplicates from a Sequence while Maintaining Order

Eliminate the duplicate values in a sequence, but preserve the order of the remaining items

In [93]:
def dedupe(items):
    seen = set()
    for item in items:
        if item not in seen:
            yield item
            seen.add(item)

In [94]:
a = [1, 5, 2, 1, 9, 1, 5, 10]
set_a = set(a)
duplicates_ordered = list(dedupe(a))

In [95]:
print("Original list: ", a)
print("Set of original list: ", set_a)
print("Ordered set of original list: ", duplicates_ordered)

Original list:  [1, 5, 2, 1, 9, 1, 5, 10]
Set of original list:  {1, 2, 5, 9, 10}
Ordered set of original list:  [1, 5, 2, 9, 10]


In [96]:
# If you are trying to eliminate duplicates in a sequence of unhashable types (such as dicts), 
# you can make a slight change to this recipe, as follows:

def dedupe(items, key=None):
    seen = set()
    for item in items:
        val = item if key is None else key(item)
        if val not in seen:
            yield item
            seen.add(val)

In [103]:
b = [ {'x':1, 'y':2}, {'x':1, 'y':3}, {'x':1, 'y':2}, {'x':2, 'y':4}]

list(dedupe(b, key=lambda d: (d['x'],d['y'])))

[{'x': 1, 'y': 2}, {'x': 1, 'y': 3}, {'x': 2, 'y': 4}]

## Problem 17: Determining the Most Frequently Occurring Items in a Sequence

Given a sequence of items, determine the most frequently occurring items in the sequence

In [104]:
words = [
   'look', 'into', 'my', 'eyes', 'look', 'into', 'my', 'eyes',
   'the', 'eyes', 'the', 'eyes', 'the', 'eyes', 'not', 'around', 'the',
   'eyes', "don't", 'look', 'around', 'the', 'eyes', 'look', 'into',
   'my', 'eyes', "you're", 'under'
]

In [105]:
from collections import Counter

word_counts = Counter(words)
top_three = word_counts.most_common(3)
print(top_three)

[('eyes', 8), ('the', 5), ('look', 4)]


In [134]:
word_counts = {}
for word in words:
    if word not in word_counts.keys():
        word_counts[word]=1
    else:
        word_counts[word]+=1

words_counts_sorted = sorted(zip(word_counts.values(), word_counts.keys()), reverse=True)
top_three = words_counts_sorted[:3]
print(top_three)

[(8, 'eyes'), (5, 'the'), (4, 'look')]


## Problem 18: Sorting a List of Dictionaries by a Common Key

Given a list of dictionaries, sort the entries according to one or more of the dictionary values

In [135]:
rows = [
    {'fname': 'Brian', 'lname': 'Jones', 'uid': 1003},
    {'fname': 'David', 'lname': 'Beazley', 'uid': 1002},
    {'fname': 'John', 'lname': 'Cleese', 'uid': 1001},
    {'fname': 'Big', 'lname': 'Jones', 'uid': 1004}
]

In [137]:
from operator import itemgetter

rows_by_fname = sorted(rows, key=itemgetter('fname'))
print(rows_by_fname)

[{'fname': 'Big', 'lname': 'Jones', 'uid': 1004}, {'fname': 'Brian', 'lname': 'Jones', 'uid': 1003}, {'fname': 'David', 'lname': 'Beazley', 'uid': 1002}, {'fname': 'John', 'lname': 'Cleese', 'uid': 1001}]


In [138]:
rows_by_uid = sorted(rows, key=itemgetter('uid'))
print(rows_by_uid)

[{'fname': 'John', 'lname': 'Cleese', 'uid': 1001}, {'fname': 'David', 'lname': 'Beazley', 'uid': 1002}, {'fname': 'Brian', 'lname': 'Jones', 'uid': 1003}, {'fname': 'Big', 'lname': 'Jones', 'uid': 1004}]


In [139]:
rows_by_lfname = sorted(rows, key=itemgetter('lname','fname'))
print(rows_by_lfname)

[{'fname': 'David', 'lname': 'Beazley', 'uid': 1002}, {'fname': 'John', 'lname': 'Cleese', 'uid': 1001}, {'fname': 'Big', 'lname': 'Jones', 'uid': 1004}, {'fname': 'Brian', 'lname': 'Jones', 'uid': 1003}]


## Problem 19: Sorting Objects Without Native Comparison Support

Sort objects of the same class, but they don’t natively support comparison operations

In [140]:
class User(object):
    def __init__(self, user_id):
        self.user_id = user_id
    def __repr__(self):
        return 'User({})'.format(self.user_id)

In [142]:
users = [User(23), User(3), User(99)]
users

[User(23), User(3), User(99)]

In [143]:
sorted(users, key=lambda u: u.user_id) 

[User(3), User(23), User(99)]

In [144]:
from operator import attrgetter
sorted(users, key=attrgetter('user_id')) 

[User(3), User(23), User(99)]

In [145]:
min(users, key=lambda u: u.user_id) 

User(3)

In [146]:
max(users, key=lambda u: u.user_id) 

User(99)

## Problem 20: Grouping Records Together Based on a Field
Given a sequence of dictionaries or instances and you want to iterate over the data in groups based on the value of a particular field, such as date.

In [147]:
rows = [
    {'address': '5412 N CLARK', 'date': '07/01/2012'},
    {'address': '5148 N CLARK', 'date': '07/04/2012'},
    {'address': '5800 E 58TH', 'date': '07/02/2012'},
    {'address': '2122 N CLARK', 'date': '07/03/2012'},
    {'address': '5645 N RAVENSWOOD', 'date': '07/02/2012'},
    {'address': '1060 W ADDISON', 'date': '07/02/2012'},
    {'address': '4801 N BROADWAY', 'date': '07/01/2012'},
    {'address': '1039 W GRANVILLE', 'date': '07/04/2012'},
]

In [151]:
from operator import itemgetter
from itertools import groupby

# Sort by the desired field first
rows.sort(key=itemgetter('date'))
print(rows)

[{'address': '5412 N CLARK', 'date': '07/01/2012'}, {'address': '4801 N BROADWAY', 'date': '07/01/2012'}, {'address': '5800 E 58TH', 'date': '07/02/2012'}, {'address': '5645 N RAVENSWOOD', 'date': '07/02/2012'}, {'address': '1060 W ADDISON', 'date': '07/02/2012'}, {'address': '2122 N CLARK', 'date': '07/03/2012'}, {'address': '5148 N CLARK', 'date': '07/04/2012'}, {'address': '1039 W GRANVILLE', 'date': '07/04/2012'}]


In [149]:
# Iterate in groups
for date, items in groupby(rows, key=itemgetter('date')):
    print(date)
    for i in items:
        print('    ', i)

07/01/2012
     {'address': '5412 N CLARK', 'date': '07/01/2012'}
     {'address': '4801 N BROADWAY', 'date': '07/01/2012'}
07/02/2012
     {'address': '5800 E 58TH', 'date': '07/02/2012'}
     {'address': '5645 N RAVENSWOOD', 'date': '07/02/2012'}
     {'address': '1060 W ADDISON', 'date': '07/02/2012'}
07/03/2012
     {'address': '2122 N CLARK', 'date': '07/03/2012'}
07/04/2012
     {'address': '5148 N CLARK', 'date': '07/04/2012'}
     {'address': '1039 W GRANVILLE', 'date': '07/04/2012'}


## Problem 21: Filtering Sequence Elements

Given a sequence of data, extract values or reduce the sequence using some criteria

In [152]:
mylist = [1, 4, -5, 10, -7, 2, 3, -1]

In [155]:
comprehension_positives = [n for n in mylist if n > 0] 
comprehension_positives

[1, 4, 10, 2, 3]

In [156]:
generator_positives = (n for n in mylist if n > 0)
for x in generator_positives:
    print(x)

1
4
10
2
3


In [157]:
values = ['1', '2', '-3', '-', '4', 'N/A', '5']

def is_int(val):
    try:
        x = int(val)
        return True
    except ValueError:
        return False

ivals = list(filter(is_int, values))
print(ivals) 

['1', '2', '-3', '4', '5']


## Problem 22: Test for Palindromic Permutations
Write a program to test whether the letters forming a string can be permuted to form a palindrome. For example, 'edified' can be permuted to form 'deified'

In [7]:
from collections import Counter

# the TRICK here is to realize that a string can be permuted to form a palindrome if and only if
# the number of chars whose frequencies is odd is at most 1

def can_form_palindrome(my_string):
    return sum(x % 2 for x in Counter(my_string).values())<=1

In [8]:
print(can_form_palindrome('edified'))
print(can_form_palindrome('level'))
print(can_form_palindrome('rotator'))

True
True
True


In [9]:
Counter('level')

Counter({'l': 2, 'e': 2, 'v': 1})

## Problem 23: Compute the Intersection of Two Sorted Arrays
Write a program which takes as input two sorted arrays, and returns a new array containins elements that present in both

In [13]:
# Brute force solution: a 'loop join'of O(n*m) complexity

def intersec (A, B):
    return [a for i, a in enumerate (A) if (i==0 or a!=A[i-1]) and a in B]

In [14]:
x = [2,3,3,5,5,6,7,7,8,12]
y = [5,5,6,8,8,9,10,10]

intersec(x, y)

[5, 6, 8]

In [10]:
# Linear time solution O(n+m): by simultaneously advancing throught the two arrays in increasing order

def intersec(A, B):
    i, j, intersection_AB = 0, 0, []
    while i < len(A) and j < len(B):
        if A[i] == B[j]:
            if i == 0 or A[i]!=A[i-1]:
                intersection_AB.append(A[i])
            i, j = i+1, j+1
        elif A[i] < B[j]:
            i += 1 
        else:
            j += 1
    return intersection_AB

In [12]:
x = [2,3,3,5,5,6,7,7,8,12]
y = [5,5,6,8,8,9,10,10]

intersec(x, y)

[5, 6, 8]

## Problem 24: Kadene's algorithm (another classic)
 Write a program that takes an array denoting the daily stock price, and returns the maximum profit that could be made by buying and then selling one share of the stock

In [15]:
def buy_and_sell_stock(prices):
    min_price_so_far, max_profit = float('inf'), 0.0
    for price in prices:
        max_profit_sell_today = price - min_price_so_far
        max_profit = max(max_profit, max_profit_sell_today)
        min_price_so_far = min(min_price_so_far, price)
    return max_profit

In [16]:
prices = [310, 315, 275, 295, 260, 270, 290, 230, 255, 250]
buy_and_sell_stock(prices)

30

## Problem 25: Merge two Sorted List
Write a program that takes two lists, assumed to be sorted and returns their merge.

In [21]:
test_list1 = [1, 2, 5, 6, 9 ] 
test_list2 = [3, 4, 7, 8, 10] 
  
# printing original lists  
print ("The original list 1 is : " + str(test_list1)) 
print ("The original list 2 is : " + str(test_list2)) 

The original list 1 is : [1, 2, 5, 6, 9]
The original list 2 is : [3, 4, 7, 8, 10]


In [22]:
def merge_sorted_lists(test_list1, test_list2):
    size_1 = len(test_list1) 
    size_2 = len(test_list2) 
    res = [] 
    i, j = 0, 0

    while i < size_1 and j < size_2: 
        if test_list1[i] < test_list2[j]: 
            res.append(test_list1[i]) 
            i += 1
        else: 
            res.append(test_list2[j]) 
            j += 1

    res = res + test_list1[i:] + test_list2[j:] 
    return res

In [23]:
  
print ("The combined sorted list is : " + str(merge_sorted_lists(test_list1, test_list2))) 

The combined sorted list is : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


## Problem 26: Root Square of the Conditional Values of a dictionary 

Write a list comprenhension that return the root square of the values of a dictionary conditional to his key is an integer

In [44]:
mydict = {'a':123, 12.123:120, '01':1241, 0:121, 1:144}

In [45]:
[v**.5 for k, v in mydict.items() if isinstance(k, int)]

[11.0, 12.0]

## Problem 27: More about Palindromes

Determine whether an integer is a palindrome. An integer is a palindrome when it reads the same backward as forward.

In [1]:
def isPalindrome( x: int) -> bool:
    if x < 0 or (x % 10 ==0 and x != 0):
        return False
    reverted_number = 0
    num = x
    while x > reverted_number:
        reverted_number = reverted_number * 10 + num % 10
        num = int(num/10)
        
    return x == reverted_number or x == reverted_number/10

In [7]:
isPalindrome(987656789)

True

In [3]:
isPalindrome(11)

True

## Problem 28: Reversing integers 

Given a 32-bit signed integer, reverse digits of an integer. Assume we are dealing with an environment which could only store integers within the 32-bit signed integer range: (−2^31,  2^31 - 1). For the purpose of this problem, assume that your function returns 0 when the reversed integer overflows.

In [4]:
def reverse(x):
    INT_MAX = 2 ** 31 - 1
    INT_MIN = -2 ** 31
    if x >= 0 :
        sol = int(str(x)[::-1])
        if (sol > INT_MAX ):
            return 0
        else:
            return sol
    else: 
        sol = int(str(abs(x))[::-1])*(-1)
        if (sol < INT_MIN) :
            return 0
        else:
            return sol

In [5]:
reverse(-123)

-321

In [6]:
reverse(120)

21

## Problem 29: Find two elements that sum equal the target...

Given an array of integers, return indices of the two numbers such that they add up to a specific target.

You may assume that each input would have exactly one solution, and you may not use the same element twice.

In [9]:
# A 'brut-force' solution

def TwoSum(nums, target):
    for i, num in enumerate(nums):
        rest_nums = nums[i+1:]
        if (target - num) in rest_nums :
            return [i, rest_nums.index(target - num) + i + 1] 
    return []

In [10]:
twoSum([0,4,3,0], 0)

[0, 3]

In [11]:
twoSum([-1,-2,-3,-4,-5], -8)

[2, 4]

In [12]:
twoSum([-18,12,3,0], -6)

[0, 1]

## Problem 30: Count the number of ways to traverse a 2D array

How many ways can you go from the top-left to bottom-right in a 2D (n,m)-array, if all the movements must either go right or down.

Solution should be O(nm) in time complexity and space complexity

In [48]:
def number_of_ways(n: int,m: int) -> int:
    def compute_number_of_ways_to_xy(x,y):
        if x == y == 0: 
            return 1
        if number_of_ways[x][y] == 0:
            ways_top = 0 if x == 0 else compute_number_of_ways_to_xy(x-1, y) # coming from the right
            ways_left = 0 if y == 0 else compute_number_of_ways_to_xy(x, y-1) # coming from the top
            number_of_ways[x][y] = ways_top + ways_left # caching in a matrix
        return number_of_ways[x][y]
    
    number_of_ways = [[0] * m for _ in range(n)]
    return compute_number_of_ways_to_xy(n-1, m-1)
            

In [49]:
number_of_ways(3,3)

6

## Problem 31: Find the Longest Common Prefix
Write a function to find the longest common prefix string amongst an array of strings.

If there is no common prefix, return an empty string "".

In [6]:
def longestCommonPrefix(strs):
    if not strs: return ""
    if len(strs)==0: return ""
    if len(strs)==1: return strs[0]
    min_len = min(map(lambda x:len(x), strs))
    if min_len == 0 : return ""
    matrix = [[char for char in word] for word in strs ]
    for j in range(min_len):
        for i in range(len(matrix)-1):
            if matrix[i][j]==matrix[i+1][j]:
                pass
            else:
                return strs[i][:j]
    return strs[i][:min_len]

In [7]:
strs = ["abab", "aba", "abc"]
longestCommonPrefix(strs)

'ab'

In [8]:
strs = ["abca","aba","aaab"]
longestCommonPrefix(strs)

'a'

In [9]:
strs = ["flower","flow","flight"]
longestCommonPrefix(strs)

'fl'